In [1]:
import torch
from torch import nn
import pickle as pkl
import numpy as np

In [2]:
filehandler = open("data.pkl","rb")
dataset = pkl.load(filehandler)
filehandler.close()

In [3]:
class Conv3DFeatureDetector(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        
        size_1 = 100
        size_2 = 200
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, size_1, kernel_size=(kernel_size, kernel_size, kernel_size), padding= (kernel_size - 1)//2),
            nn.Tanh(),
#             nn.MaxPool3d((2, 2, 2)),
        )
        self.fc1 = nn.Linear(size_1, size_2)
        self.relu = nn.Tanh()
        self.fc2 = nn.Linear(size_2, 1)
        
    def forward(self, x):
        out = self.conv1(x)
        out = out.permute(0,2,3,4,1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = torch.sigmoid(out)
        return out

In [4]:
X = np.vstack([x[None,None,:,:,:] for (x,y) in dataset])
X = X.astype(np.float32)
# X[X == 0.0] = -1.0
print(X.shape)

Y = np.vstack([y[None,:,:,:,None] for (x,y) in dataset])
Y = Y.astype(np.float32)
print(Y.shape)

dev = torch.device("cuda")

model = Conv3DFeatureDetector(9)
model.to(dev)

error = nn.BCELoss()

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.95))

X_train = torch.from_numpy(X[:500]).to(dev)
Y_train = torch.from_numpy(Y[:500]).to(dev)
# X_test = torch.from_numpy(X[500:]).to(dev)
# Y_test = torch.from_numpy(Y[500:]).to(dev)

print(X.shape)
o = model(X_train[1:10])
print(o.shape)

(1000, 1, 25, 25, 22)
(1000, 25, 25, 22, 1)
(1000, 1, 25, 25, 22)
torch.Size([9, 25, 25, 22, 1])


In [ ]:
batch_size = 10

for epoch in range(1000):
#     permutation = torch.randperm(X_torch.size()[0])
    permutation = torch.arange(X_train.size()[0])
    
    print("epoch ", epoch)
    total_loss = 0.0
    for i in range(0,X_train.size()[0], batch_size):
#         print(i)
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], Y_train[indices]

        # in case you wanted a semi-full example
        outputs = model.forward(batch_x)
        batch_y.cuda()
        
        loss = (outputs - batch_y)**2
        loss[batch_y == 1.0] *= 100.0
        loss = loss.mean()
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print("total loss: ", total_loss)

epoch  0
total loss:  38.15052002668381
epoch  1
total loss:  29.014645040035248
epoch  2
total loss:  23.73376974463463
epoch  3
total loss:  21.94264307618141
epoch  4
total loss:  21.03337585926056
epoch  5
total loss:  20.410024017095566
epoch  6
total loss:  19.939922481775284
epoch  7
total loss:  19.579494178295135
epoch  8
total loss:  19.301514238119125
epoch  9
total loss:  19.083242684602737
epoch  10
total loss:  18.90803959965706
epoch  11
total loss:  18.764511287212372
epoch  12


In [21]:
torch.save(model.state_dict(), "mug_handle_model.torch")

In [15]:
test_X = X_train[-40:]

In [16]:
pred_Y = model.forward(test_X)

In [17]:
pred_Y

tensor([[[[[0.0372],
           [0.0372],
           [0.0372],
           ...,
           [0.0372],
           [0.0372],
           [0.0372]],

          [[0.0372],
           [0.0372],
           [0.0372],
           ...,
           [0.0372],
           [0.0372],
           [0.0372]],

          [[0.0372],
           [0.0372],
           [0.0372],
           ...,
           [0.0372],
           [0.0372],
           [0.0372]],

          ...,

          [[0.0372],
           [0.0372],
           [0.0372],
           ...,
           [0.0372],
           [0.0372],
           [0.0372]],

          [[0.0372],
           [0.0372],
           [0.0372],
           ...,
           [0.0372],
           [0.0372],
           [0.0372]],

          [[0.0372],
           [0.0372],
           [0.0372],
           ...,
           [0.0372],
           [0.0372],
           [0.0372]]],


         [[[0.0372],
           [0.0372],
           [0.0372],
           ...,
           [0.0372],
           [0.0372

In [18]:
pred_Y[Y[-100:] == 1.0]

IndexError: The shape of the mask [100, 27, 27, 26, 1] at index 0 does not match the shape of the indexed tensor [40, 27, 27, 26, 1] at index 0

In [19]:
pred_Y[Y[-100:] == 0.0]

IndexError: The shape of the mask [100, 27, 27, 26, 1] at index 0 does not match the shape of the indexed tensor [40, 27, 27, 26, 1] at index 0

In [20]:
with open("result.pkl","wb") as f:
    pkl.dump((test_X.cpu().detach().numpy(), pred_Y.cpu().detach().numpy()), f)